# Alzheimer's Simulation Integration Tests

This notebook verifies the causal chain in the vivarium Alzheimer's simulation:

1. **Testing coverage** at key time points (2030, 2035) matches spec
2. **Testing → Treatment**: positive BBBM tests drive treatment initiation
3. **Treatment → Progression**: treatment slows BBBM → MCI transition

### Spec reference (PR 1888)

**BBBM testing rate** (piecewise-linear propensity threshold):
- (2027.0, 0%) → (2030.5, 10%) → (2045.0, 50%) → (2055.0, 60%)

**Treatment probability** (piecewise-linear propensity threshold):
- (2027.0, 0%) → (2035.5, 30%) → (2100.0, 80%)

**Positive diagnosis probability**: 50%

In [ ]:
import pandas as pd
import numpy as np
from vivarium import InteractiveContext

SPEC_PATH = '../src/vivarium_csu_alzheimers/model_specifications/model_spec.yaml'
POPULATION_SIZE = 10_000

# Disease state names
BBBM_STATE = 'alzheimers_blood_based_biomarker_state'
MCI_STATE = 'alzheimers_mild_cognitive_impairment_state'
DEMENTIA_STATE = 'alzheimers_disease_state'
DISEASE_COL = 'alzheimers_disease_and_other_dementias'

## Helper functions

In [ ]:
def step_to_year(sim, target_year, target_month=7):
    """Step the simulation until we reach the target year and month."""
    target = pd.Timestamp(f'{target_year}-{target_month:02d}-01')
    steps = 0
    while sim.current_time < target:
        sim.step()
        steps += 1
    print(f'Stepped {steps} times to reach {sim.current_time} (target: {target})')
    return sim


def get_eligible_for_bbbm(pop):
    """Return mask of simulants eligible for BBBM testing.
    
    Eligibility: BBBM state, age 65-80, no prior positive test, alive.
    (Ignoring retesting interval for this summary check.)
    """
    return (
        (pop[DISEASE_COL] == BBBM_STATE)
        & (pop['age'] >= 65)
        & (pop['age'] < 80)
        & (pop['bbbm_test_result'] != 'positive')
        & (pop['alive'] == 'alive')
    )


def testing_summary(pop, label=''):
    """Print a summary of testing state for the population."""
    eligible = get_eligible_for_bbbm(pop)
    n_eligible = eligible.sum()
    n_total = len(pop[pop['alive'] == 'alive'])
    
    # Among eligible: how many have been tested (bbbm_test_date is not NaT)?
    tested_ever = eligible & pop['bbbm_test_date'].notna()
    n_tested = tested_ever.sum()
    
    # Among all alive: BBBM test results
    results = pop.loc[pop['alive'] == 'alive', 'bbbm_test_result'].value_counts()
    
    print(f'\n=== {label} ===')
    print(f'Alive simulants: {n_total}')
    print(f'Eligible for BBBM test (BBBM state, age 65-80, not positive): {n_eligible}')
    print(f'Eligible & ever tested: {n_tested} ({n_tested/n_eligible*100:.1f}% of eligible)' if n_eligible > 0 else 'No eligible simulants')
    print(f'BBBM test results (all alive):\n{results.to_string()}')
    return {'n_eligible': n_eligible, 'n_tested': n_tested, 'n_total': n_total}


def treatment_summary(pop, label=''):
    """Print a summary of treatment state for the population."""
    alive = pop['alive'] == 'alive'
    treatment_states = pop.loc[alive, 'treatment'].value_counts()
    
    # Among BBBM-positive: what fraction entered treatment?
    positive = alive & (pop['bbbm_test_result'] == 'positive')
    n_positive = positive.sum()
    
    treatment_active_states = [
        'waiting_for_treatment', 'treatment_effect',
        'waning_effect', 'no_effect_after_treatment'
    ]
    in_treatment = positive & pop['treatment'].isin(treatment_active_states)
    n_in_treatment = in_treatment.sum()
    
    declined = positive & (pop['treatment'] == 'no_effect_never_treated')
    n_declined = declined.sum()
    
    print(f'\n=== {label} ===')
    print(f'Treatment states (all alive):\n{treatment_states.to_string()}')
    print(f'\nBBBM-positive simulants: {n_positive}')
    if n_positive > 0:
        print(f'  In treatment pipeline: {n_in_treatment} ({n_in_treatment/n_positive*100:.1f}%)')
        print(f'  Declined treatment:    {n_declined} ({n_declined/n_positive*100:.1f}%)')
        print(f'  Still susceptible:     {n_positive - n_in_treatment - n_declined}')
    return {'n_positive': n_positive, 'n_in_treatment': n_in_treatment, 'n_declined': n_declined}


def disease_summary(pop, label=''):
    """Print a summary of disease states."""
    alive = pop['alive'] == 'alive'
    states = pop.loc[alive, DISEASE_COL].value_counts()
    print(f'\n=== {label} ===')
    print(f'Disease states (alive):\n{states.to_string()}')
    return states.to_dict()

---
## Test 1: BBBM Testing Coverage at 2030 and 2035

**What we expect:**
- The BBBM testing rate is a propensity threshold: simulants with `testing_propensity < rate(t)` get tested
- At mid-2030: rate = 10%, so ~10% of eligible simulants should have propensity below threshold
- At mid-2035: rate is interpolated between (2030.5, 10%) and (2045.0, 50%)
  - 2035.5 is 5 years into a 14.5-year ramp → rate ≈ 10% + 40% × (5/14.5) ≈ **23.8%**
- The **actual** fraction tested will be lower than the rate because:
  - Historical test assignment at init avoids immediate testing surges
  - Retesting intervals (3-5 years) mean not everyone eligible is due for a test
  - Some eligible simulants just entered eligibility and haven't had a chance yet

In [ ]:
# Create simulation with BBBM testing + treatment enabled
sim = InteractiveContext(
    SPEC_PATH,
    configuration={
        'population': {'population_size': POPULATION_SIZE},
        'intervention': {'scenario': 'bbbm_testing_and_treatment'},
    }
)
print(f'Simulation initialized at {sim.current_time}')
pop = sim.get_population()
print(f'Initial population: {len(pop)} simulants')
testing_summary(pop, 'Initial state (2022)')

In [ ]:
# Step to mid-2030
step_to_year(sim, 2030, target_month=7)
pop_2030 = sim.get_population()
stats_2030 = testing_summary(pop_2030, f'Testing at {sim.current_time.date()} (target: mid-2030)')

In [ ]:
# Detailed check: among eligible simulants, compare propensity to testing rate
eligible_mask = get_eligible_for_bbbm(pop_2030)
eligible = pop_2030[eligible_mask].copy()

# The testing rate at this time should be ~10%
# Check: what fraction of eligible simulants with propensity < 0.10 have been tested?
EXPECTED_RATE_2030 = 0.10
low_propensity = eligible[eligible['testing_propensity'] < EXPECTED_RATE_2030]
high_propensity = eligible[eligible['testing_propensity'] >= EXPECTED_RATE_2030]

low_tested = low_propensity['bbbm_test_date'].notna().sum()
high_tested = high_propensity['bbbm_test_date'].notna().sum()

print(f'Testing rate at {sim.current_time.date()}: expected ~{EXPECTED_RATE_2030:.0%}')
print(f'\nEligible with propensity < {EXPECTED_RATE_2030}: {len(low_propensity)}')
print(f'  Of those, ever BBBM-tested: {low_tested} ({low_tested/len(low_propensity)*100:.1f}%)' if len(low_propensity) > 0 else '  None')
print(f'\nEligible with propensity >= {EXPECTED_RATE_2030}: {len(high_propensity)}')
print(f'  Of those, ever BBBM-tested: {high_tested} ({high_tested/len(high_propensity)*100:.1f}%)' if len(high_propensity) > 0 else '  None')
print(f'\n→ Low-propensity simulants should be mostly tested (may be <100% due to retesting interval)')
print(f'→ High-propensity simulants should NOT be tested (expect 0%)')

In [ ]:
# Step to mid-2035
step_to_year(sim, 2035, target_month=7)
pop_2035 = sim.get_population()
stats_2035 = testing_summary(pop_2035, f'Testing at {sim.current_time.date()} (target: mid-2035)')

In [ ]:
# Same propensity check at 2035
# Expected rate at 2035.5: interpolated between (2030.5, 10%) and (2045.0, 50%)
EXPECTED_RATE_2035 = 0.10 + 0.40 * (5.0 / 14.5)  # ~23.8%

eligible_mask = get_eligible_for_bbbm(pop_2035)
eligible = pop_2035[eligible_mask].copy()

low_propensity = eligible[eligible['testing_propensity'] < EXPECTED_RATE_2035]
high_propensity = eligible[eligible['testing_propensity'] >= EXPECTED_RATE_2035]

low_tested = low_propensity['bbbm_test_date'].notna().sum()
high_tested = high_propensity['bbbm_test_date'].notna().sum()

print(f'Testing rate at {sim.current_time.date()}: expected ~{EXPECTED_RATE_2035:.1%}')
print(f'\nEligible with propensity < {EXPECTED_RATE_2035:.3f}: {len(low_propensity)}')
print(f'  Of those, ever BBBM-tested: {low_tested} ({low_tested/len(low_propensity)*100:.1f}%)' if len(low_propensity) > 0 else '  None')
print(f'\nEligible with propensity >= {EXPECTED_RATE_2035:.3f}: {len(high_propensity)}')
print(f'  Of those, ever BBBM-tested: {high_tested} ({high_tested/len(high_propensity)*100:.1f}%)' if len(high_propensity) > 0 else '  None')

---
## Test 2: Testing Drives Treatment

**What we expect:**
- Only BBBM-positive simulants should enter the treatment pipeline
- At mid-2035, treatment probability is ~30%
- So ~30% of BBBM-positive simulants should be in treatment states
  (waiting, treatment_effect, waning, no_effect_after_treatment)
- The rest should be in `no_effect_never_treated`
- No simulant without a positive test should be in treatment

In [ ]:
# Treatment summary at 2035 (using pop_2035 from above)
stats_treatment_2035 = treatment_summary(pop_2035, f'Treatment at {sim.current_time.date()}')

In [ ]:
# Verify: no one without a positive test should be in treatment
alive = pop_2035['alive'] == 'alive'
not_positive = pop_2035['bbbm_test_result'] != 'positive'
treatment_states = [
    'waiting_for_treatment', 'treatment_effect',
    'waning_effect', 'no_effect_after_treatment', 'no_effect_never_treated'
]
wrongly_treated = alive & not_positive & pop_2035['treatment'].isin(treatment_states)
n_wrong = wrongly_treated.sum()
print(f'Simulants in treatment states WITHOUT positive BBBM test: {n_wrong}')
print(f'→ This should be 0' if n_wrong == 0 else f'→ ERROR: {n_wrong} simulants incorrectly in treatment!')

In [ ]:
# Check treatment acceptance rate among BBBM-positive
# Expected: ~30% at mid-2035 (from treatment probability ramp)
positive = alive & (pop_2035['bbbm_test_result'] == 'positive')
positive_pop = pop_2035[positive]

EXPECTED_TREATMENT_RATE_2035 = 0.30
low_prop = positive_pop[positive_pop['treatment_propensity'] < EXPECTED_TREATMENT_RATE_2035]
high_prop = positive_pop[positive_pop['treatment_propensity'] >= EXPECTED_TREATMENT_RATE_2035]

accepted_treatment = [
    'waiting_for_treatment', 'treatment_effect',
    'waning_effect', 'no_effect_after_treatment'
]

low_in_treatment = low_prop['treatment'].isin(accepted_treatment).sum()
high_in_treatment = high_prop['treatment'].isin(accepted_treatment).sum()

print(f'Treatment probability at {sim.current_time.date()}: expected ~{EXPECTED_TREATMENT_RATE_2035:.0%}')
print(f'\nBBBM-positive with treatment_propensity < {EXPECTED_TREATMENT_RATE_2035}: {len(low_prop)}')
print(f'  In treatment pipeline: {low_in_treatment} ({low_in_treatment/len(low_prop)*100:.1f}%)' if len(low_prop) > 0 else '  None')
print(f'\nBBBM-positive with treatment_propensity >= {EXPECTED_TREATMENT_RATE_2035}: {len(high_prop)}')
print(f'  In treatment pipeline: {high_in_treatment} ({high_in_treatment/len(high_prop)*100:.1f}%)' if len(high_prop) > 0 else '  None')
print(f'\n→ Low-propensity should be mostly in treatment pipeline')
print(f'→ High-propensity should mostly be in no_effect_never_treated')

---
## Test 3: Treatment Reduces BBBM → MCI Progression

**What we expect:**
- Treatment applies a relative risk (RR ≈ 0.4-0.6) to the BBBM→MCI transition rate
- To see this effect, we compare two scenarios at the same time point:
  - `bbbm_testing` (testing only, no treatment)
  - `bbbm_testing_and_treatment` (testing + treatment)
- The treatment scenario should have fewer MCI transitions
- We run to 2045 for a stronger signal (higher testing/treatment rates by then)

In [ ]:
# Continue treatment sim to 2045 for stronger signal
step_to_year(sim, 2045, target_month=1)
pop_treat_2045 = sim.get_population()
disease_treat = disease_summary(pop_treat_2045, f'Treatment scenario at {sim.current_time.date()}')
treatment_summary(pop_treat_2045, f'Treatment states at {sim.current_time.date()}')

In [ ]:
# Now run a testing-only scenario (no treatment) to the same point
print('Setting up testing-only scenario for comparison...')
sim_testing = InteractiveContext(
    SPEC_PATH,
    configuration={
        'population': {'population_size': POPULATION_SIZE},
        'intervention': {'scenario': 'bbbm_testing'},
    }
)
step_to_year(sim_testing, 2045, target_month=1)
pop_test_2045 = sim_testing.get_population()
disease_test = disease_summary(pop_test_2045, f'Testing-only scenario at {sim_testing.current_time.date()}')

In [ ]:
# Compare disease state distributions
print('\n=== Scenario Comparison at 2045 ===')
print(f'{"State":<50} {"Testing Only":>15} {"Testing+Treatment":>18}')
print('-' * 85)
for state in [BBBM_STATE, MCI_STATE, DEMENTIA_STATE]:
    n_test = disease_test.get(state, 0)
    n_treat = disease_treat.get(state, 0)
    short_name = state.replace('alzheimers_', '').replace('_state', '')
    print(f'{short_name:<50} {n_test:>15} {n_treat:>18}')

# Check: treatment scenario should have more BBBM (stayed longer) and fewer MCI
mci_test = disease_test.get(MCI_STATE, 0)
mci_treat = disease_treat.get(MCI_STATE, 0)
bbbm_test = disease_test.get(BBBM_STATE, 0)
bbbm_treat = disease_treat.get(BBBM_STATE, 0)

print(f'\nMCI count difference: {mci_treat - mci_test} (treatment - testing_only)')
print(f'BBBM count difference: {bbbm_treat - bbbm_test} (treatment - testing_only)')
print(f'\n→ Treatment should reduce MCI (negative difference) and increase BBBM (positive difference)')
print(f'→ MCI reduced: {"YES ✓" if mci_treat < mci_test else "NO — check further"}')
print(f'→ BBBM retained: {"YES ✓" if bbbm_treat > bbbm_test else "NO — check further"}')

---
## Test 4: Baseline Sanity Check

**What we expect:**
- In the baseline scenario (no BBBM testing, no treatment):
  - No BBBM tests should occur
  - All simulants should remain in `susceptible_to_treatment`
  - CSF/PET baseline testing should still work for MCI/Dementia simulants

In [ ]:
# Run baseline scenario
print('Setting up baseline scenario...')
sim_baseline = InteractiveContext(
    SPEC_PATH,
    configuration={
        'population': {'population_size': POPULATION_SIZE},
        'intervention': {'scenario': 'baseline'},
    }
)
step_to_year(sim_baseline, 2035, target_month=7)
pop_baseline = sim_baseline.get_population()

alive_bl = pop_baseline['alive'] == 'alive'
print(f'\nBaseline at {sim_baseline.current_time.date()}:')

# No BBBM tests
bbbm_tested = (pop_baseline.loc[alive_bl, 'bbbm_test_result'] != 'not_tested').sum()
print(f'Simulants with BBBM tests: {bbbm_tested} (should be 0)')

# All in susceptible treatment state
treatment_states = pop_baseline.loc[alive_bl, 'treatment'].value_counts()
print(f'\nTreatment states:\n{treatment_states.to_string()}')
non_susceptible = (pop_baseline.loc[alive_bl, 'treatment'] != 'susceptible_to_treatment').sum()
print(f'\nSimulants in non-susceptible treatment: {non_susceptible} (should be 0)')

# CSF/PET testing should still work
csf_pet = pop_baseline.loc[alive_bl, 'testing_state'].value_counts()
print(f'\nCSF/PET testing (should be nonzero for MCI/Dementia):\n{csf_pet.to_string()}')

---
## Summary

| Check | What to look for |
|---|---|
| Testing at 2030 | ~10% of eligible have propensity below threshold; those are tested |
| Testing at 2035 | ~24% threshold; tested fraction growing |
| Testing → Treatment | Only BBBM-positive enter treatment; ~30% acceptance at 2035 |
| Treatment → Progression | Fewer MCI in treatment scenario vs testing-only |
| Baseline sanity | No BBBM tests, no treatment, CSF/PET still works |